In [2]:
import urllib.request
import json
import pandas as pd
import time
KEY_API = '7VGM2ZKZ92VRXI4IPDIMXFC2CPA3A1HH2M'


def getAccountTransaction(account: str):
    f = urllib.request.urlopen(
        'https://api-ropsten.etherscan.io/api?module=account&action=tokentx&address=' + account +
        '&startblock=0&endblock=999999999&sort=asc&apikey=' + KEY_API
    )
    time.sleep(1)
    data = json.loads(f.read().decode('utf-8'))
    return data['result']


def getData(accountId: str) -> list:
    outputData = []
    toAccount = []
    transaction = getAccountTransaction(accountId.lower())
    for i in transaction:
        if (i['from'] == accountId.lower() and i['tokenSymbol'] == 'BKTC'):
            outputData.append({
                'txHash': i['hash'],
                'from': i['from'],
                'to': i['to'],
                'value': int(i['value'])
            })
            toAccount.append(i['to'])
    toAccount = list(dict.fromkeys(toAccount))  # remove dup
    if (len(toAccount) > 0):
        for account in toAccount:
            outputData.extend(getData(account)) # recursive
    return outputData



In [3]:
global CSVoutput
CSVoutput = getData('0xEcA19B1a87442b0c25801B809bf567A6ca87B1da')
pd.DataFrame(CSVoutput).to_csv('output.csv')


In [4]:
df = pd.DataFrame(CSVoutput)

In [8]:
df['value'] = df['value'] / 10**18

In [9]:
accountBalance = df.groupby('to').sum()

In [10]:
accountBalance.loc[:, ['value']]

,value
to,
0x04bb39f36bbadf4e31aa1df09a2f5a0b76114a62,2.0
0x11759e580b444aa1136482929984c76a19ef2794,3.0
0x4b41947e269464408bbd9dce7346a7e07d522b12,5.0
0x635893f9918cbb41cb6c6bde4a87e5959f53dd56,5.0
0x711447b288e2f853fa9716aa8eba37b02ca8a50f,13.0
0x873ce5c27fe1a679b724e25b5a856282a93b137f,5.0
0x8d16abfba10c079340ddf2b252f94353cdccfa57,11.0
0x8f8d0e8c921b6fbee6da796f3c823a8a1e354bea,6.0
0x936146f6850dc157e77fc060c442e0bdffbbe6ff,7.0
